<a href="https://colab.research.google.com/github/nematatu/Colab-Python/blob/master/%E6%84%9F%E6%83%85%E3%81%AB%E3%82%88%E3%82%8B%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%81%AE%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
#ファイルの作成、ディレクトリの移動とか、OS依存の機能をいじるのに使う
import os
#re(Regular Expressions)正規表現を扱うのに使う
#文字列パターンを検索、抽出、置換したり
import re
#ファイル、ディレクトリの操作をするのに使う
#osと何が違うんだ↓
#shutilはosを基に、高度なファイル、ディレクトリ操作ができるようになったものらしい
import shutil
#文字列の操作に使う
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
print(tf.__version__)

2.14.0


In [3]:
#IMDB(Internet Movie DataBase)という50000件の映画レビューから抽出した、データセットを使う
#トレーニング用とテスト用に25000件ずつある
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
#↑urlからファイルをダウンロードする
#tf.keras.utils.get_file()
#"aclImdb": 保存先のディレクトリ名
#url: urlの指定
#untar: 保存するファイルが.tarなら、解凍するかどうか
#cache_dir: ファイルのダウンロード先のディレクトリを指定
#cache_subdir: 保存先のサブディレクトリを指定。今回はなし
dataset=tf.keras.utils.get_file("aclImdb",url,untar=True,cache_dir='.',cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset),'aclImdb')

84125825/84125825 [==============================] - 12s 0us/step


In [4]:
print(os.listdir(os.path.dirname(dataset)))
os.listdir(dataset_dir)

['.config', 'aclImdb.tar.gz', 'aclImdb', 'sample_data']


['README', 'train', 'test', 'imdb.vocab', 'imdbEr.txt']

In [5]:
train_dir=os.path.join(dataset_dir,'train')
print(os.listdir(train_dir))

['unsupBow.feat', 'unsup', 'labeledBow.feat', 'pos', 'urls_pos.txt', 'neg', 'urls_neg.txt', 'urls_unsup.txt']


In [6]:
sample_file=os.path.join(train_dir,'pos/1189_9.txt')
#with: 安全にファイル操作ができるらしい
with open(sample_file) as f:
  print(f.read())

The Last Hunt is one of the few westerns ever made to deal with Buffalo hunting, both as a sport and business and as a method of winning the plains Indian wars. Before the white man set foot on the other side of the Mississippi, the plains used to have herds of American Bison as large as some of our largest cities. By the time of the period The Last Hunt is set in, the buffalo had been all but wiped out. The 20th century, due to the efforts of conservationists, saw a revival in population of the species, but not hardly like it once was.<br /><br />Robert Taylor and Stewart Granger are co-starring in a second film together and this one is far superior to All the Brothers Were Valiant. Here Stewart Granger is the good guy, a world weary buffalo hunter, who has to go back to a job he hates because of financial considerations.<br /><br />The partner he's chosen to throw in with is Robert Taylor. Forgetting Taylor for the moment, I doubt if there's ever been a meaner, nastier soul than Char

In [7]:
remove_dir=os.path.join(train_dir,'unsup')
shutil.rmtree(remove_dir)

In [8]:
batch_size=32
seed=42

#元のディレクトリから、テキスト形式のトレーニング用データセットにする
raw_train_ds=tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    #内20%を検証用に使う
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [9]:
print(raw_train_ds.class_names)

['neg', 'pos']


In [10]:
for text_batch,label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review",text_batch.numpy()[i])
    print("Label",label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [11]:
print("label 0 correspond to ",raw_train_ds.class_names[0])
print("label 1 correspond to ",raw_train_ds.class_names[1])

label 0 correspond to  neg
label 1 correspond to  pos


In [12]:
#検証用データセットを作成
raw_val_ds=tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [13]:
#テスト用データセットを作成
raw_test_ds=tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size
)

Found 25000 files belonging to 2 classes.


In [34]:
#<br />とかのHTMLタグを消す編
def custom_standardization(input_data):
  lower_case=tf.strings.lower(input_data)
  #lower_case: 入力された文字列を小文字にしたもの
  #の中から、<br />を検索して、空白に置き換える
  stripped_html=tf.strings.regex_replace(lower_case,'<br />',' ')
  #string.panctuationに句読点などが含まれていて、それを/などをつけてエスケープしている
  #エスケープするとは、/!とすることで!に意味を持たせないようにすること
  #なぜそうするかというと、正規表現内で!は意味があるらしく、めんどくさいことを回避するためにやってるらしい
  #そうして集めた奴らを、['%s']の%sに代入して空白に置換している
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),' ')

In [35]:
#データの前処理編
max_features=10000
sequence_length=250

#テキストデータを整数のベクトルに変換しているらしい
vectorize_layer=layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [36]:
#前処理実践編
#.map()でデータセットに関数を適応させる
#lambdaは無名関数で、xとyを引数にとってxを返す関数
#raw_train_dsは、(text,label)の形になっていて、textだけを抽出しているということ
train_text=raw_train_ds.map(lambda x,y:x)
#前処理として、レイヤーにトレーニングデータを元に学習させているだけ
#関数の引数的なvectorize_layer(train_text)とは違う
#↑はちゃんとテキストをベクトルに変換している
#↓は変換まではしない
vectorize_layer.adapt(train_text)

In [ ]:
#前処理確認編
def vectorize_text(text,label):
  #textの要素の-1番目、すなわち最後に１つ次元を追加する
  #(batch_size,sequence_length)->(batch_size,sequence_length,1)となる
  #最初に追加したければ、ext=tf.expand_dims(text,0)
  text=tf.expand_dims(text,-1)
